https://proglib.io/p/fun-nlp/


# Facts extraction pipeline

Рассмотрим типичный процесс извлечения фактов из текста на примере вот такого отрывка:

> London is the capital and most populous city of England and the United Kingdom. Standing on the River Thames in the south east of the island of Great Britain, London has been a major settlement for two millennia. It was founded by the Romans, who named it Londinium.

Чтобы извлечь факты нам потребуется проделать следующие шаги:

## Шаг 1. Выделить предложения

Как правило, каждое предложение содержит законченную мысль или идею. В то же время, проще научить понимать компьютер единственное предложение, а не целый параграф. Можно было бы просто разделять текст по определенным знакам препинания. Но современные NLP пайплайны имеют в запасе более сложные методы, подходящие даже для работы с неформатированными фрагментами.

## Шаг 2. Выполнить токенцизацию

Далее мы должны разделить предложения на состовные части - токены. В данном случае, токенами будут слова и знаки препинания.

> London is the capital and most populous city of England and the United Kingdom.

После токенизации становится `['London', 'is', 'the', 'capital', 'and', 'most', 'populous', 'city', 'of', 'England', 'and', 'the', 'United', 'Kingdom', '.']`

## Шаг 3. Определить части речи

Теперь посмотрим на каждый токен и постараемся угадать, какой частью речи он является: существительным, глаголом, прилагательным или чем-то другим. Зная роль каждого слова в предложении, можно понять его общий смысл. Для этого необходима модель, обученная на корпусе текстов, в данном случае, английского языка. Модель пытается определить, какой частью речи является каждый отдельно взятый токен. Причем для определения используется не единственные токен, а 1-3 соседних токена.

Такие модели имеют исключительно статистическую подоплёку, поэтому говорить о том, что модель "понимает" смысл слов, чтобы определить часть речь - ну такое.

## Шаг 4. Выполнить лемматизацию

Лемматизация - это процесс нахождения *основной формы* слова. Например, глаголы прошедшего времени превратить в инфинитивы. Леммой существительного, использованного в тексте во множественном числе, будет являться то же существительное, только в единственном числе. Лемматизация, так же, делается при помощи модели, обученной на корпусе языка.

## Шаг 5. Выделить стоп-слова

Теперь мы хотим определить важность каждого слова в предложении. В английском очень много вспомогательных слов, например, «and», «the», «a». При статистическом анализе текста эти токены создают много шума, так как появляются чаще, чем остальные. Некоторые NLP пайплайны отмечают их как стоп-слова и отсеивают перед подсчетом количества. 

Для обнаружения стоп-слов обычно используются готовые таблицы. Однако нет единого стандартного списка, подходящего в любой ситуации. Игнорируемые токены могут меняться, все зависит от особенностей проекта.

## Шаг 6. Распарсить зависимости
Теперь необходимо установить взаимосвязь между словами в предложении. Это называется парсингом зависимостей. Конечная цель этого шага – построение дерева, в котором каждый токен имеет единственного родителя. Корнем может быть главный глагол.

Нужно не только определить родителя, но и установить тип связи между двумя словами:
<center><img src='img/dependencies.png'></center>

Парсинг зависимостей так же выполняется при помощи моделей. По состоянию на 2016 год лучшим парсером был Гугловский ParseySaurus. Мы же воспользуемся spaCy.

## Шаг 6'. Сгруппировать существительные:
*NB: Данный шаг не является обязательным, однако стоит вспомнить о нём, если вместо максимально подробной информации о словах мы стремимся извлечь законченные идеи.*


Иногда имеет смысл сгруппировать токены, которые относятся к одной и той же идее или вещи. Мы можем использовать полученное дерево парсинга, чтобы автоматически объединить такие слова.

Вместо токенизации по словам:
<center><img src='img/before.png'></center>

Группировкой существительных можно добиться повышения плотности смысловой нагрузки на токен:
<center><img src='img/after.png'></center>

## Шаг 7. Распознавать именованные сущности (Named Entity Recognition, NER):

Цель распознавания именованных сущностей – обнаружить такие существительные и связать их с реальными концепциями. Делается это снова при помощи моделей, обученных на корпусах текстов, где сущности уже размечены. После обработки каждого токена NER-моделью наше предложение будет выглядеть вот так:

<center><img src='img/NER.png'></center>

NER-системы не просто просматривают словари. Они анализируют контекст токена в предложении и используют статистические модели, чтобы угадать какой объект он представляет. Хорошие NER-системы способны отличить актрису Brooklyn Decker от города Brooklyn.

Большинство NER-моделей распознают следующие типы объектов:

- Имена людей
- Названия компаний
- Географические обозначения (и физические, и политические)
- Продукты
- Даты и время
- Денежные суммы
- События

## Шаг 8. Разрешить кореференции

Теперь выйдем за пределы одного предложения. Вот мы распарсили предложения, распознали сущности в предложениях:

> London has been a major settlement for two millennia. It was founded by the Romans, who named it Londinium.

А теперь посмотрим на второе предложение. 

> **It** was founded by the Romans, who named **it** Londinium

**Оно** было основано римлянами. Это предложение не содержит информацию о том, что это за **оно**. А вот предыдущее содержит. Чтобы извлечь факт "Лондон был основан римлянами" придётся связать предложение с предыдущим. Такой процесс называется *разрешением кореференций*. 

Формальное определение: *разрешением кореференции называется отслеживание местоимений в предложениях с целью выбрать все слова, относящиеся к одной сущности*.

## Итог

В итоге имеем такой пайплайн:
1. Сегментация предложений
1. Токенизация
1. Определение частей речи
1. Лемматизация
1. Выделение стоп-слов
1. Парсинг зависимостей (+ группировка существительных)
1. Распознование именованных сущностей
1. Разрешение кореференций

# Имплементация на python

NLP в Python представлен библиотекой `spacy`. Два других инструмента `textacy` для извлечения фактов и `neuralcoref` для разрешения кореференций являются надстройками для `spacy`.

Установим библиотеки:

In [ ]:
# !pip install spacy==2.0.11
# !pythom -m spacy download en_core_web_md 

# !pip install textacy

# !pip install neuralcoref
# !pip install https://github.com/huggingface/neuralcoref-models/releases/download/en_coref_md-3.0.0/en_coref_md-3.0.0.tar.gz

In [104]:
import spacy

# Загрузка английской NLP-модели
engine = spacy.load('en_core_web_lg')

spacy.lang.en.English

Посмотрим пайплайн модели:

In [103]:
for part in engine.pipeline:
    print(part)

('tagger', <spacy.pipeline.Tagger object at 0x7f781e0380f0>)
('parser', <spacy.pipeline.DependencyParser object at 0x7f790d27b048>)
('ner', <spacy.pipeline.EntityRecognizer object at 0x7f7982224c50>)


- `Tagger` выделяет части речи и проводит лемматизацию
- `Parser` парсит зависимости
- `ner` выделяет именованные сущности

In [68]:
# Текст для анализа
text = """London is the capital and most populous city of England and 
the United Kingdom.  Standing on the River Thames in the south east 
of the island of Great Britain, London has been a major settlement 
for two millennia. It was founded by the Romans, who named it Londinium.
"""

# Обработка
doc = engine(text)

# Посмотрим, какие сущности выделила модель:
for entity in doc.ents:
    print(f"{entity.text} ({entity.label_})")

London (GPE)
England (GPE)
the United Kingdom (GPE)
the River Thames (FAC)
Great Britain (GPE)
London (GPE)
two millennia (DATE)
Romans (NORP)
Londinium (PERSON)


Категории сущностей, распознаные NPE-движком из spacy:

- `GPE` - Countries, cities, states.
- `FAC` - Buildings, airports, highways, bridges, etc.
- `DATE` - Absolute or relative dates or periods.
- `NORP` - Nationalities or religious or political groups.
- `PERSON` - People, including fictional.

С полным списком категорий [можно ознакомиться здесь](https://spacy.io/usage/linguistic-features#entity-types)

Теперь извлечём факты. Для этого нам потребуется модуль `extract` библиотеки `textacy` и его функция `semistructured_statements`.

In [52]:
# Воспользуемся развёрнутым текстом

text = """London is the capital of Great Britain, its political, economic and cultural centre. It's one of the largest cities in the world. Its population is more than million people. London is situated on the river Thames. The city is very old and beautiful. It was founded more than two thousand years ago. Traditionally London is divided into several parts: the City, the West End, the East End and Westminster. The City is the oldest part of London, its financial and business centre. The heart of the City is the Stock Exchange. Westminster is the most important part of the capital. It's the administrative centre. The Houses of Parliament, the seat of the British Government, are there. It's a very beautiful building with two towers and a very big clock called Big Ben. Big Ben is really the bell which strikes every quarter of an hour. Opposite the Houses of Parliament is Westminster Abbey. It's a very beautiful church built over 900 years ago. The tombs of many great statesmen, scientists and writers are there.

To the west of Westminster is West End. Here we find most of the big shops, hotels, museums, art galleries, theatres and concert halls. Picadilly Circus is the heart of London's West End. In the West End there are wide streets with beautiful houses and many parks, gardens and squares. To the east of Westminster is the East End, an industrial district of the capital. There are no parks or gardens in the East End and you can't see many fine houses there. Most of the plants and factories are situated there. London has many places of interest. One of them is Buckingham Palace. It's the residence of the Queen. The English are proud of Trafalgar Square, which was named so in memory of the victory at the battle. There in 1805 the English fleet defeated the fleet of France and Spain. The last place of interest I should like to mention, is the British Museum, the biggest museum in London. The museum is famous for its library -one of the richest in the world.

All London's long-past history is told by its streets. There are many streets in London which are known all over the" world. Among them Oxford Street, Downing Street and a lot of others can be mentioned. And tourists are usually attracted not only by the places of interest but by the streets too. In conclusion I should say if you are lucky enough to find yourself in London some day you will have a lot to see and enjoy there."""



In [30]:
import spacy
import textacy.extract as extract

engine = spacy.load('en_core_web_md')

In [31]:
# Анализ
doc = engine(text)
 
# Извлечение фактов со словом London
statements = extract.semistructured_statements(doc, "London")
  
for statement in statements:
    subject, verb, fact = statement
    print(f"{subject} / {verb} / {fact}")

London / is / the capital of Great Britain, its political, economic and cultural centre


Теперь выполним разрешение кореференций. Для этого потребуется `neuralcoref` и одна из обученных на нём моделей. 

In [14]:
import en_coref_md
import textacy.extract as extract

neural_engine = en_coref_md.load()

Посмотрим на пайплайн новой модели:

In [36]:
for part in neural_engine.pipeline:
    print(part)

('tagger', <spacy.pipeline.Tagger object at 0x7f5ec2139198>)
('parser', <spacy.pipeline.DependencyParser object at 0x7f5ec21eec50>)
('ner', <spacy.pipeline.EntityRecognizer object at 0x7f5ec21ee8e0>)
('neuralcoref', <en_coref_md.neuralcoref.neuralcoref.NeuralCoref object at 0x7f5e52122ac8>)


Она отличается компонентом `neuralcoref` который отвечает за разрешение кореференции (напоминаю, связывание местоимений с сущностями, упомянутыми ранее). Процесс обработки текста выполняетс точно так же:

In [55]:
doc = neural_engine(text)

Можем посмотреть все найденные кореференции:

In [53]:
doc._.coref_clusters

[London: [London, London, The city, It, London, The City, London, its, the City],
 Great Britain: [Great Britain, Great Britain],
 Londons: [Londons, It, Its],
 Westminster: [Westminster, Westminster, It],
 Big Ben: [Big Ben, Big Ben],
 Opposite the Houses of Parliament: [Opposite the Houses of Parliament, It],
 West End: [West End, West End, the West End],
 London: [London, London],
 the East End: [the East End, the East End],
 many places of interest: [many places of interest, them],
 the British Museum, the biggest museum in London: [the British Museum, the biggest museum in London, The museum, its],
 London: [London, London, London, London],
 All London's long-past history: [All London's long-past history, its],
 its streets: [its streets, the streets],
 many streets in London which are known all over the" world: [many streets in London which are known all over the" world, them]]

А так же версию текста, где местоимения уже заменены соответствующими сущностями:

In [57]:
doc._.coref_resolved

'London is the capital of Great Britain, London political, economic and cultural centre. London\'s one of the largest cities in the world. London population is more than million people. London is situated on the river Thames. London is very old and beautiful. London was founded more than two thousand years ago. Traditionally London is divided into several parts: the City, the West End, the East End and Westminster. London is the oldest part of London, London financial and business centre. The heart of London is the Stock Exchange. Westminster is the most important part of the capital. Westminster\'s the administrative centre. The Houses of Parliament, the seat of the British Government, are there. It\'s a very beautiful building with two towers and a very big clock called Big Ben. Big Ben is really the bell which strikes every quarter of an hour. Opposite the Houses of Parliament is Westminster Abbey. Opposite the Houses of Parliament\'s a very beautiful church built over 900 years ago

Внимательный глаз заметит, что разрешение кореференций не идеально.

Попробуем извлечь факты из нового текста:

In [58]:
doc = neural_engine(text)
resolved_doc = neural_engine(doc._.coref_resolved)  # Обработаем новый текст

for document, label in zip([doc, resolved_doc], ['Before resolving', 'After resolving']):
    statements = extract.semistructured_statements(document, "London")
    print(label)
    for statement in statements:
        subject, verb, fact = statement
        print(f"{subject} / {verb} / {fact}")
    
    print('')

Before resolving
London / is / the capital of Great Britain, its political, economic and cultural centre

After resolving
London / is / the capital of Great Britain, London political, economic and cultural centre
London / is / very old and beautiful
London / is / the oldest part of London, London financial and business centre



Итого пара новых фактов.